In [6]:
# STEP 1 - Install ipywidgets

# Install and enable ipywidgets for interactive form UI in Google Colab
!pip install ipywidgets --quiet
!jupyter nbextension enable --py widgetsnbextension --sys-prefix > /dev/null 2>&1

In [7]:
# STEP 2 - Backend Logic

import os
import json
from datetime import datetime

# Ensure the directory exists for task storage
os.makedirs("/mnt/data", exist_ok=True)

# Path to the JSON file where tasks will be stored
TASK_FILE = "/mnt/data/tasks.json"

# Load all tasks from the JSON file
def load_tasks():
    if not os.path.exists(TASK_FILE):
        return []
    with open(TASK_FILE, "r") as f:
        try:
            return json.load(f)
        except json.JSONDecodeError:
            return []

# Save all tasks to the JSON file
def save_tasks(tasks):
    with open(TASK_FILE, "w") as f:
        json.dump(tasks, f, indent=4)

# Generate the next unique task ID
def get_next_id(tasks):
    return max([task["id"] for task in tasks], default=0) + 1

# Add a new task
def add_task(description):
    tasks = load_tasks()
    now = datetime.now().isoformat()
    task = {
        "id": get_next_id(tasks),
        "description": description,
        "status": "todo",
        "createdAt": now,
        "updatedAt": now
    }
    tasks.append(task)
    save_tasks(tasks)
    return f"✅ Task added successfully (ID: {task['id']})"

# Update an existing task's description
def update_task(task_id, new_description):
    tasks = load_tasks()
    for task in tasks:
        if task["id"] == task_id:
            task["description"] = new_description
            task["updatedAt"] = datetime.now().isoformat()
            save_tasks(tasks)
            return f"✏️ Task {task_id} updated successfully."
    return f"❌ Task with ID {task_id} not found."

# Delete a task by ID
def delete_task(task_id):
    tasks = load_tasks()
    updated_tasks = [t for t in tasks if t["id"] != task_id]
    if len(tasks) == len(updated_tasks):
        return f"❌ Task with ID {task_id} not found."
    save_tasks(updated_tasks)
    return f"🗑️ Task {task_id} deleted successfully."

# Mark a task as in-progress or done
def mark_status(task_id, status):
    tasks = load_tasks()
    for task in tasks:
        if task["id"] == task_id:
            task["status"] = status
            task["updatedAt"] = datetime.now().isoformat()
            save_tasks(tasks)
            return f"🔄 Task {task_id} marked as '{status}'."
    return f"❌ Task with ID {task_id} not found."

# List all tasks or filter by status
def list_tasks(filter_status=None):
    tasks = load_tasks()
    if filter_status:
        tasks = [t for t in tasks if t["status"] == filter_status]
    return tasks

In [8]:
# STEP 3 - Interactive UI using ipywidgets

import ipywidgets as widgets
from IPython.display import display, clear_output

# Input fields and buttons for the UI
desc_input = widgets.Text(description="Task:")
add_btn = widgets.Button(description="Add Task", button_style='success')

task_id_input = widgets.IntText(description="Task ID:")
new_desc_input = widgets.Text(description="New Desc:")
update_btn = widgets.Button(description="Update Task", button_style='warning')

delete_btn = widgets.Button(description="Delete Task", button_style='danger')

status_dropdown = widgets.Dropdown(
    options=["todo", "in-progress", "done"],
    value="in-progress",
    description="Mark as:"
)
mark_btn = widgets.Button(description="Mark Status", button_style='info')

list_filter = widgets.Dropdown(
    options=["all", "todo", "in-progress", "done"],
    value="all",
    description="Show:"
)
list_btn = widgets.Button(description="List Tasks")

output = widgets.Output()  # Output area to show messages and task lists

# --- Callback Functions ---

# Add Task
def on_add_clicked(b):
    with output:
        clear_output()
        print(add_task(desc_input.value))

# Update Task
def on_update_clicked(b):
    with output:
        clear_output()
        print(update_task(task_id_input.value, new_desc_input.value))

# Delete Task
def on_delete_clicked(b):
    with output:
        clear_output()
        print(delete_task(task_id_input.value))

# Mark Task as Done or In Progress
def on_mark_clicked(b):
    with output:
        clear_output()
        print(mark_status(task_id_input.value, status_dropdown.value))

# List Tasks
def on_list_clicked(b):
    with output:
        clear_output()
        status = None if list_filter.value == "all" else list_filter.value
        tasks = list_tasks(status)
        if not tasks:
            print("📭 No tasks found.")
        for task in tasks:
            print(f"📝 ID: {task['id']} | [{task['status'].upper()}] {task['description']} \n📅 Created: {task['createdAt']} | Updated: {task['updatedAt']}\n")

# Bind buttons to their respective handlers
add_btn.on_click(on_add_clicked)
update_btn.on_click(on_update_clicked)
delete_btn.on_click(on_delete_clicked)
mark_btn.on_click(on_mark_clicked)
list_btn.on_click(on_list_clicked)

# Display the full UI in a vertical layout
display(widgets.VBox([
    widgets.HBox([desc_input, add_btn]),
    widgets.HBox([task_id_input, new_desc_input, update_btn]),
    widgets.HBox([task_id_input, delete_btn]),
    widgets.HBox([task_id_input, status_dropdown, mark_btn]),
    widgets.HBox([list_filter, list_btn]),
    output
]))